# Positive/negativeコメントを品詞で見る

Date: 2023/12/3

In [1]:
import spacy

!python3 -m spacy download "ja_core_news_sm"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 1.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('ja_core_news_sm')


In [2]:
import yaml

with open('../data/impressions.yaml', 'r') as f:
    imp = yaml.safe_load(f.read())

In [3]:
nlp = spacy.load('ja_core_news_sm')
imp.keys()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dict_keys(['negative', 'positive', 'positive_negative', 'random'])

In [13]:
imp_ = []
for doc in nlp.pipe(imp['positive_negative']):
    parts = []
    prev = 0
    imp__ = []
    for idx, token in enumerate(doc):
        #print(f'{token.text}[{idx}]({token.pos_}) ', end='')
        if token.pos_ == 'SCONJ' and token.text in ('が'):
            parts.append([prev, idx+1])
            prev = idx+1
    for p0, p1 in parts:
        #print(doc[p0:p1].text.strip('、'), end=': ')
        imp__.append(doc[p0:p1].text.strip('、'))
    if len(parts) > 0:
        last_p = parts[-1][1]
        if last_p < len(doc):
            #print(doc[last_p: len(doc)].text.strip('、'))
            imp__.append(doc[last_p: len(doc)].text.strip('、'))
        else:
            print()
    imp_.append(imp__)
print(imp_)

[['ショールームの清潔さには好感が持てましたが', '一部の展示品が傷んでいるのが気になりました。'], ['スタッフの丁寧な対応がありましたが', '商品の説明が少なく、理解しにくかったです。'], ['展示品の品質は高く、特にデザインに魅了されましたが', '価格が高すぎると感じました。'], ['ショールームの広さとレイアウトは素晴らしかったが', '一部のエリアが混雑していました。'], [], ['インテリアデザインがモダンで印象的でしたが', '一部のエリアが古く感じられました。'], ['スタッフの対応は親切で好感が持てましたが', '混雑しておりゆっくり見学できませんでした。'], ['展示品のディスプレイは美しく、高い品質が感じられましたが', '価格が高すぎると感じました。'], ['ショールーム内の照明が良好で、商品が見やすかったが', '一部のエリアが薄暗かったです。'], ['商品の価格設定が適切で良心的でしたが', 'スタッフの知識が不足しているように感じました。'], ['展示品の配置が工夫され、テーマ性がありましたが', '一部の商品が見つけにくかったです。'], ['ショールーム内の案内が分かりやすく、スタッフの対応も良かったが', '混雑が気になりました。'], ['商品の新しさとデザインに魅了されましたが', 'スタッフの態度が冷淡で残念でした。'], ['ショールームの雰囲気が明るく、居心地が良かったが', '一部の商品が陳腐に感じられました。'], ['展示品の保守状態が良好で、商品の信頼性に安心感を覚えましたが', '価格が心配です。'], ['ショールームの配置が効果的で見やすかったが', 'スタッフの忙しさが対応に影響していました。'], ['スタッフの知識が豊富で的確でしたが', '商品の新しさに欠け、刷新が必要かもしれません。'], ['商品の魅力が引き立つディスプレイでしたが', 'スタッフの説明が簡略化されていた印象です。'], ['ショールーム内の音楽が心地よく、雰囲気が良かったが', '一部エリアが騒がしかったです。'], ['展示品のデザインが洗練されており、一部の商品には本当に惹かれましたが', '価格が心配です。'], ['ショールームの温度が快適で、滞在が楽しかったが', '一部のエリアが寒かった

In [40]:
# 色々とモデルを試した方が良い。

from transformers import pipeline
#sentiment = pipeline(model='christian-phu/bert-finetuned-japanese-sentiment')
sentiment = pipeline(model='llm-book/bert-base-japanese-v3-marc_ja')

In [41]:
for i in imp_:
    print(','.join(i))
    print(sentiment(','.join(i)))
    print(i)
    print(sentiment(i))
    print()

ショールームの清潔さには好感が持てましたが,一部の展示品が傷んでいるのが気になりました。
[{'label': 'negative', 'score': 0.9413160085678101}]
['ショールームの清潔さには好感が持てましたが', '一部の展示品が傷んでいるのが気になりました。']
[{'label': 'negative', 'score': 0.9102018475532532}, {'label': 'negative', 'score': 0.9673207998275757}]

スタッフの丁寧な対応がありましたが,商品の説明が少なく、理解しにくかったです。
[{'label': 'negative', 'score': 0.9560932517051697}]
['スタッフの丁寧な対応がありましたが', '商品の説明が少なく、理解しにくかったです。']
[{'label': 'negative', 'score': 0.8465325236320496}, {'label': 'negative', 'score': 0.9913062453269958}]

展示品の品質は高く、特にデザインに魅了されましたが,価格が高すぎると感じました。
[{'label': 'negative', 'score': 0.7011358737945557}]
['展示品の品質は高く、特にデザインに魅了されましたが', '価格が高すぎると感じました。']
[{'label': 'positive', 'score': 0.9985824823379517}, {'label': 'negative', 'score': 0.9912781119346619}]

ショールームの広さとレイアウトは素晴らしかったが,一部のエリアが混雑していました。
[{'label': 'positive', 'score': 0.9428516626358032}]
['ショールームの広さとレイアウトは素晴らしかったが', '一部のエリアが混雑していました。']
[{'label': 'positive', 'score': 0.8677330613136292}, {'label': 'negative', 'score': 0.8609099984169006}]

